# Analysis Part I - Preprocessing Sample 3

In [ ]:
%load_ext autoreload

In [ ]:
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore')

In [ ]:
import os
import scanpy as sc
import scirpy as ir
import anndata as ann
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib import rcParams
from mudata import MuData
import mudata
import tarfile
import warnings
from glob import glob
import muon as mu

In [ ]:
%autoreload 2
import sys
sys.path.append('..')
import utility.annotation as utils_annotation
import utility.representation as utils_representation
import utility.visualisation as utils_vis

In [ ]:
sc.settings.set_figure_params(dpi=150)
sc.settings.verbosity = 3
sc.set_figure_params(vector_friendly=True, color_map='viridis', transparent=True)
sb.set_style('whitegrid')

Samples:

- Sample 3:
    - C12 + controldex
    - C5
    - B11_d14 + controldex
    - B11_1yr
    - B19_d14
    - B19_1yr + controldex
    - B34_d15 + controldex
    - B34_1yr
    
Dextrameres:
- NS4B214-222 -- LLWNGPMAV (A*02:01) -- TTGGCGATTCCTCCA
- NS2B117-125 -- VLAGWLFHV (A*02:01) -- AAGCTAGGAGCATAC
- NS3293-301 -- FLDPASIAA (A*02:01) -- GCACATATATCTTGA
- NS3286-294 -- IIMDEAHFL (A*02:01) -- ACAGGAACACCAGTG
- NS324-32 -- IYGIFQSTF (A*24:02) -- CAGTTCGACTCTTCC
- NS5672-680 -- RPIDDRFGL (B*07:02) -- ACGGCTTATCGGTTG
- NSA97-106 -- SPRERLVLTL (B*07:02) -- GCGGAGAATATTGCT
- NS4B165-173 -- ALYEKKLAL (B*08:01) -- GAGTAACGCCGTGAT

Controls:
- SARS-CoV-2 -- LTDEMIAQY (A*01:01) -- GGTTCGACGCATACC
- HHV-1 -- ATDSLNNEY (A*01:01) -- AATATGCCGGCGGAT
- Flu-A -- CTELKLSDY (A*01:01) -- CGCCATTCGCTCGGT
- EBV1 -- FLRGRAYGL (B*08:01) -- TAATTGCTGAGGCCT
- EBV2 -- RAKFKQLL (B*08:01) -- TGATGCGTAAGCGAA

In [ ]:
#Define the lists for later
hashtags = [f'sample{i}' for i in range(1, 9)]

epitope_ids = ['NS4B214', 'NS2B117', 'NS3293', 'NS3286', 'NS324', 'NS5672',
               'NS2A97', 'NS4B165', 'COV', 'HHV', 'FLU', 'EBV1', 'EBV2']

cite_seqs = ['CD45RA', 'CCR7-1', 'CD95', 'CD62L', 'CXCR3-1', 'CD27']

feature_barcode_ids = hashtags + epitope_ids + cite_seqs

In [ ]:
##Read data

# GEX data
datafile = "/media/agschober/HDD12/3_scRNA-Seq_Sina/Cellranger_output/1st_Experiment/CS3-multi_new2/outs/per_sample_outs/CS3-multi_new2/count/sample_filtered_feature_bc_matrix.h5"
adata = sc.read_10x_h5(datafile, gex_only=False)
adata.var_names_make_unique()

# VDJ data
adata_vdj = ir.io.read_10x_vdj("/media/agschober/HDD12/3_scRNA-Seq_Sina/Cellranger_output/1st_Experiment/CS3-multi_new2/outs/per_sample_outs/CS3-multi_new2/vdj_t/filtered_contig_annotations.csv")
#ir.pp.merge_with_ir(adata, adata_vdj)

# Epitope data
adata.uns['epitopes'] = epitope_ids
for e in epitope_ids:
    adata.obs[e] = adata[:, e].X.A.copy()

# Hashtag data
adata.uns['hashtags'] = hashtags
for h in hashtags:
    adata.obs[h] = adata[:, h].X.A.copy()

# CiteSeq Data
adata.uns['cite_ids'] = cite_seqs
for c in cite_seqs:
    adata.obs[c] = adata[:, c].X.A.copy()
    
# Remove Barcodes from counts
adata = adata[:, [gene for gene in adata.var_names if gene not in feature_barcode_ids]]
adata.obs['sample'] = f'sample3'

adata.shape

In [ ]:
#fuse the information of gene expression and tcr
adata = mu.MuData({"gex": adata, "airr": adata_vdj})

### Quality control

Basic analysis by amount counts, genes, and fraction of mitochondrial genes

In [ ]:
adata["gex"].obs['n_counts'] = adata["gex"].X.A.sum(axis=1)
adata["gex"].obs['log_counts'] = np.log10(adata["gex"].obs['n_counts'])
adata["gex"].obs['n_genes'] = (adata["gex"].X.A > 0).sum(axis=1)
adata["gex"].obs['log_genes'] = np.log10(adata["gex"].obs['n_genes'])

mt_gene_mask = [gene.startswith('MT-') for gene in adata.var_names]
mt_gene_idx = np.where(mt_gene_mask)[0]
adata["gex"].obs['mt_frac'] = adata["gex"].X.A[:, mt_gene_idx].sum(1) / adata["gex"].X.A.sum(axis=1)

In [ ]:
print('Mean # Genes: ', adata["gex"].obs['n_genes'].mean())
print('Median # Genes: ', adata["gex"].obs['n_genes'].median())
print('Mean # Counts: ', adata["gex"].obs['n_counts'].mean())
print('Median # Counts: ', adata["gex"].obs['n_counts'].median())
print('Mean % MT: ', adata["gex"].obs['mt_frac'].mean())
print('Median % MT: ', adata["gex"].obs['mt_frac'].median())

In [ ]:
rcParams['figure.figsize'] = (4, 4)
sc.pl.violin(adata["gex"], ['n_counts'], size=1, log=False, rotation=90)
sc.pl.violin(adata["gex"], ['n_genes'], size=1, log=False, rotation=90)
sc.pl.violin(adata["gex"], ['mt_frac'], size=1, log=False, rotation=90)

- counts up to 20000, but mostly below 10000
- number of genes up to 6000, but mostly below 4000
- mitochondrial fraction up to 0.1 but mostly below 0.05

In [ ]:
rcParams['figure.figsize'] = (8, 8)
sc.pl.scatter(adata["gex"], y='n_genes', x='n_counts', color ='mt_frac', size=10, show=False)
sc.pl.scatter(adata["gex"][np.logical_and(adata["gex"].obs['n_genes']<1500, adata["gex"].obs['n_counts']<8000)],
         y='n_genes', x='n_counts', color='mt_frac', size=10, show=False)
plt.show()

In [ ]:
b = ((adata['gex'].obs['n_counts']).sort_values()).to_list()
c = ((adata['gex'].obs['n_genes']).sort_values()).to_list()

In [ ]:
plt.plot(b)
plt.ylabel('counts')
plt.xlabel('barcode')

In [ ]:
plt.plot(c)
plt.ylabel('genes')
plt.xlabel('barcode')

In [ ]:
plt.plot(b)
plt.ylabel('counts')
plt.xlabel('barcode')
plt.ylim((0,3000))
plt.xlim((0,200))

In [ ]:
plt.plot(c)
plt.ylabel('genes')
plt.xlabel('barcode')
plt.ylim((0,1000))
plt.xlim((0,100))

- remove cells with more than 4000 genes and more than 10000 counts
- remove cells with more than 0.08 mt_fraction
- remove cells with less than 800 genes and 1500 counts

### Filtering of the cells

In [ ]:
params_filter = {   'mt_frac': 0.08,
    'n_counts_min': 1500,
    'n_counts_max': 10000,
    'n_genes_min': 800,
}

In [ ]:
print(f'Size before filtering: {len(adata)}')
adata = adata[adata["gex"].obs['mt_frac'] < params_filter['mt_frac']]
adata = adata[adata["gex"].obs['n_counts'] > params_filter['n_counts_min']]
adata = adata[adata["gex"].obs['n_counts'] < params_filter['n_counts_max']]
adata = adata[adata["gex"].obs['n_genes'] > params_filter['n_genes_min']].copy()
print(f'Size after filtering: {len(adata)}')
    
adata.shape

### QC after filtering

In [ ]:
rcParams['figure.figsize'] = (4, 4)
sc.pl.violin(adata["gex"], ['n_counts'], size=1, log=False, rotation=90)
sc.pl.violin(adata["gex"], ['n_genes'], size=1, log=False, rotation=90)
sc.pl.violin(adata["gex"], ['mt_frac'], size=1, log=False, rotation=90)

rcParams['figure.figsize'] = (8, 8)
sc.pl.scatter(adata["gex"], y='n_genes', x='n_counts', color ='mt_frac', size=10, show=False)
sc.pl.scatter(adata["gex"][np.logical_and(adata["gex"].obs['n_genes']<1500, adata["gex"].obs['n_counts']<8000)],
y='n_genes', x='n_counts', color='mt_frac', size=10, show=False)
plt.show()

### TCR stats

In [ ]:
ir.pp.index_chains(adata)
ir.tl.chain_qc(adata)
adata.obs['airr:chain_pairing'].value_counts()

In [ ]:
adata.obs['airr:chain_pairing'].loc[(adata.obs['airr:chain_pairing']).isna()] = 'no_IR'

In [ ]:
adata.obs['airr:chain_pairing'].value_counts()

In [ ]:
def get_percentages_tcr(data):
    df = ir.get.airr(data, "junction_aa", ["VJ_1", "VDJ_1", "VJ_2", "VDJ_2"])
    p_alpha = df['VJ_1_junction_aa'].notnull().mean()
    p_beta = df['VDJ_1_junction_aa'].notnull().mean()
    p_paired = (df['VDJ_1_junction_aa'].notnull() & df['VJ_1_junction_aa'].notnull()).mean()
    return [p_alpha, p_beta, p_paired]

chains = ['Alpha', 'Beta', 'Paired']
percentages = get_percentages_tcr(adata)

df_tcr_fractions = {
    'chain': chains,
    'percentage': percentages
}

df_tcr_fractions = pd.DataFrame(df_tcr_fractions)
g = sb.barplot(data=df_tcr_fractions, y='percentage', x='chain')
_ = g.set_xticklabels(rotation=30, labels=chains)

### Normalise

In [ ]:
sc.pp.normalize_total(adata["gex"], target_sum=1e4)
sc.pp.log1p(adata["gex"])

### Quick Visual Sanity Check

In [ ]:
utils_representation.calculate_umap(adata["gex"], n_high_var=5000, remove_tcr_genes=True)

In [ ]:
adata["gex"].obs['chain_pairing'] = adata.obs['airr:chain_pairing']

In [ ]:
sc.pl.umap(adata["gex"])

In [ ]:
rcParams['figure.figsize'] = (6, 6)
sc.pl.umap(adata["gex"], color=['chain_pairing'])


In [ ]:
sc.pl.umap(adata["gex"], color=['n_counts', 'log_counts', 'n_genes', 'mt_frac'], ncols=2)

### Separate the samples

In [ ]:
utils_vis.distributions_over_columns(adata["gex"], hashtags, 2, 4)

In [ ]:
def hash_solo_by_sample(hashtag_cols, col_name, n_noise_barcodes):
    adata["gex"].obs[col_name] = 'NaN'

    dfs_donor = []
    adata["gex"].obs = adata["gex"].obs.drop(col_name, axis=1)
    sc.external.pp.hashsolo(adata["gex"], hashtag_cols, number_of_noise_barcodes=n_noise_barcodes)
    adata["gex"].obs = adata["gex"].obs.rename(columns={'Classification': col_name})

hash_solo_by_sample(hashtags, 'pool', 3)
adata["gex"].obs['pool'].value_counts()


In [ ]:
hash_solo_by_sample(hashtags, 'pool', 5)
adata["gex"].obs['pool'].value_counts()

In [ ]:
rcParams['figure.figsize'] = (16, 4)

for h in hashtags:
    adata["gex"].obs[f'log_{h}'] = np.log(adata["gex"].obs[h].values+1)
sb.violinplot(data=adata["gex"].obs[[f'log_{h}' for h in hashtags]], scale='area')

In [ ]:
utils_vis.adt_counts_by_condition(adata["gex"], hashtags, 'pool', 8, 4, do_log=True)

In [ ]:
rcParams['figure.figsize'] = (8, 8)
sc.pl.umap(adata["gex"], color='pool')

In [ ]:
rcParams['figure.figsize'] = (8, 8)
adata_ha = ann.AnnData(X=adata["gex"].obs[adata["gex"].uns['hashtags']], obs=adata["gex"].obs[['pool']])
adata_ha.var_names = adata["gex"].uns['hashtags']
sc.pp.log1p(adata_ha)
sc.pp.neighbors(adata_ha)
sc.tl.umap(adata_ha)
sc.pl.umap(adata_ha, color=['pool'] + [f'sample{i}' for i in range(1, 9)], ncols=3, 
           save=f'sample3_hashtag_umap.pdf')

In [ ]:
adata = adata[~adata["gex"].obs['pool'].isin(['Doublet', 'Negative'])]

### Remove Epitope Counts

In [ ]:
epitope_2_sample = {'NS4B214': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'NS2B117': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'NS3293': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'NS3286': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'NS324': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'NS5672': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'NS2A97': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'NS4B165': ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8'],
               'COV': ['sample1', 'sample3', 'sample6', 'sample7'],
               'HHV':['sample1', 'sample3', 'sample6', 'sample7'], 
               'FLU':['sample1', 'sample3', 'sample6', 'sample7'], 
               'EBV1':['sample1', 'sample3', 'sample6', 'sample7'], 
               'EBV2':['sample1', 'sample3', 'sample6', 'sample7']}

In [ ]:
for e, samples in epitope_2_sample.items():
    adata["gex"].obs.loc[~adata["gex"].obs['pool'].isin(samples), e] = np.nan

### Remove Totalseq Counts

In [ ]:
samples_full_totalseq = ['sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6', 'sample7', 'sample8']

In [ ]:
for c in cite_seqs:
    adata["gex"].obs.loc[~adata["gex"].obs['pool'].isin(samples_full_totalseq), c] = np.nan

### Save

In [ ]:
adata["gex"].obs['pool'] = f'sample3' + adata["gex"].obs['pool'].astype(str)
adata.write(filename="data3.h5mu")

In [ ]:
import session_info
session_info.show()